# Installs

In [1]:
%pip install datasets

     |████████████████████████████████| 245kB 5.2MB/s 
     |████████████████████████████████| 245kB 9.0MB/s 
     |████████████████████████████████| 122kB 10.0MB/s 


In [2]:
%pip install rouge_score

In [3]:
%pip install sacrebleu

     |████████████████████████████████| 61kB 3.4MB/s 


In [4]:
%pip install -U nltk

     |████████████████████████████████| 1.5MB 4.9MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [5]:
%pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-py2f1fsp
  Running command git clone -q https://github.com/google-research/bleurt.git /tmp/pip-req-build-py2f1fsp
     |████████████████████████████████| 358kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 35.2MB/s 
  Created wheel for BLEURT: filename=BLEURT-0.0.2-cp37-none-any.whl size=16454025 sha256=c6cfeb990248d9868cf6b048df054ed45dfb33d902fb8817cc732346546abb75
  Stored in directory: /tmp/pip-ephem-wheel-cache-ctvdcjpb/wheels/67/7a/5b/d9b15cc48bedca8bd1c444ac4d650467294980793d5dd7ba7c
Successfully built BLEURT


In [6]:
%pip install bert_score

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 2.5MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 32.1MB/s 
     |████████████████████████████████| 901kB 42.8MB/s 
  Found existing installation: huggingface-hub 0.0.13
    Uninstalling huggingface-hub-0.0.13:
      Successfully uninstalled huggingface-hub-0.0.13


In [7]:
!git clone "https://github.com/neural-dialogue-metrics/Distinct-N.git"

Cloning into 'Distinct-N'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [8]:
%pip install ./Distinct-N/

Processing ./Distinct-N
  Created wheel for Distinct-N: filename=Distinct_N-0.4.0-cp37-none-any.whl size=8788 sha256=5b481855695c270230150076bab2fee0d75810d420209cf8d98217a371150712
  Stored in directory: /root/.cache/pip/wheels/49/61/89/0086f011267cc0bb1000649e733b256cfb87ec749c446ef0bf
Successfully built Distinct-N


In [9]:
%pip install lexical-diversity

     |████████████████████████████████| 122kB 5.1MB/s 


In [10]:
!git clone https://github.com/wl-research/nubia.git

Cloning into 'nubia'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 282 (delta 79), reused 150 (delta 69), pack-reused 116
Receiving objects: 100% (282/282), 1.71 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (123/123), done.


# Load dataset

In [11]:
# Load dataset
from datasets import load_dataset
data = load_dataset("gem", "common_gen")

Dataset gem downloaded and prepared to /root/.cache/huggingface/datasets/gem/common_gen/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4. Subsequent calls will reuse this data.


## Explore dataset structure

In [14]:
data

DatasetDict({
    train: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 67389
    })
    validation: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 993
    })
    test: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 1497
    })
    challenge_train_sample: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
    challenge_validation_sample: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
    challenge_test_scramble: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
})

In [ ]:
test_instance = data['test'][0]
test_instance

In [ ]:
validation_instance = data['validation'][133]
validation_instance

In [ ]:
challenge_test_instance = data['challenge_test_scramble'][0]
challenge_test_instance

In [ ]:
challenge_validation_instance = data['challenge_validation_sample'][0]
challenge_validation_instance

In [ ]:
concept = challenge_validation_instance.get("target")
concept

In [18]:
validation_targets = []
for i in range(501): 
  instance = data['validation'][i]
  target = instance.get("target")
  validation_targets.append(target)
validation_targets[500]

'The woman rides a bicycle down the street instead of walking.'

## Extract validation split

In [12]:
validation_data = []
validation_data = data['validation']
print(validation_data['references'][0])
print(validation_data['target'][0])

['The player stood in the field looking at the batter.', 'The coach stands along the field, looking at the goalkeeper.', 'I stood and looked across the field, peacefully.', 'Someone stands, looking around the empty field.']
The player stood in the field looking at the batter.


## Clean generated text

In [13]:
# Clean validation greedy sampling generation
data_file = "sample_data/wiki_greedy_validation.txt"
greedy_generated_data = []

with open(data_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    example = line.split()
    new_example = []
    for word in example:
      if word.startswith("[unused") or word.startswith("[UNK]") or word.startswith(".") or word.startswith("\\") or word.startswith("##"):
        del word
      else:
        new_example.append(word)

    complete_sentence = " ".join(new_example) + "."
    greedy_generated_data.append(complete_sentence)

greedy_generated_data[0]

'he also stated that on what the whole field did not have to look like me , but stand up ! !.'

In [14]:
# Clean validation top-k generation
data_file = "sample_data/wiki_sampling_validation.txt"
sampling_generated_data = []

with open(data_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    example = line.split()
    new_example = []
    for word in example:
      if word.startswith("[unused") or word.startswith("[UNK]") or word.startswith(".") or word.startswith("\\") or word.startswith("##"):
        del word
      else:
        new_example.append(word)

    complete_sentence = " ".join(new_example) + "."
    sampling_generated_data.append(complete_sentence)

sampling_generated_data[0]

'he was the first pitch at the field that did not have to look like a stadium , and could stand up.'

# Compute metrics


## Lexical (ROUGE 1/2/L, BLEU, Meteor), 

In [16]:
from datasets import list_metrics, load_metric
import numpy as np

metrics_list = list_metrics()
print(metrics_list)

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'comet', 'coval', 'cuad', 'f1', 'gleu', 'glue', 'indic_glue', 'matthews_correlation', 'meteor', 'pearsonr', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'wer', 'xnli']


In [17]:
rouge_scorer = load_metric("rouge")
sacrebleu_scorer = load_metric("sacrebleu")
meteor_scorer = load_metric("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [18]:
# ROUGE target
greedy_rouge_results = rouge_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target'],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    use_agregator=True, use_stemmer=False,
)

print("Greedy rouge1", round(greedy_rouge_results['rouge1'].mid.fmeasure, 4))
print("Greedy rouge2", round(greedy_rouge_results['rouge2'].mid.fmeasure, 4))
print("Greedy rougeL", round(greedy_rouge_results['rougeL'].mid.fmeasure, 4))


sampling_rouge_results = rouge_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target'],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    use_agregator=True, use_stemmer=False,
)

print("Sampling rouge1", round(sampling_rouge_results['rouge1'].mid.fmeasure, 4))
print("Sampling rouge2", round(sampling_rouge_results['rouge2'].mid.fmeasure, 4))
print("Sampling rougeL", round(sampling_rouge_results['rougeL'].mid.fmeasure, 4))

Greedy rouge1 0.3287
Greedy rouge2 0.0712
Greedy rougeL 0.2364
Sampling rouge1 0.3093
Sampling rouge2 0.0624
Sampling rougeL 0.2182


In [19]:
greedy_rouge_results

{'rouge1': AggregateScore(low=Score(precision=0.24941439539803012, recall=0.4877388945431864, fmeasure=0.32213051145157234), mid=Score(precision=0.25477415676516324, recall=0.49850560041036873, fmeasure=0.32865631680403273), high=Score(precision=0.2600069271004271, recall=0.5097875905149466, fmeasure=0.3348074323207816)),
 'rouge2': AggregateScore(low=Score(precision=0.05068718599079398, recall=0.10096249000795912, fmeasure=0.0660402352316918), mid=Score(precision=0.05478421529385902, recall=0.10895510798839936, fmeasure=0.07120369469788296), high=Score(precision=0.05846031940753195, recall=0.116554158380492, fmeasure=0.07595059217660213)),
 'rougeL': AggregateScore(low=Score(precision=0.17862291128872632, recall=0.35172324204817657, fmeasure=0.23130483509781857), mid=Score(precision=0.182869038085204, recall=0.36043142791824656, fmeasure=0.2364130796617835), high=Score(precision=0.18719687080436181, recall=0.36990021655041766, fmeasure=0.2418221565940115))}

In [20]:
# BLEU target

greedy_bleu_results = []
for i, sentence in enumerate(zip(greedy_generated_data, validation_data)):
  greedy_bleu_result = sacrebleu_scorer.compute(
      predictions=[greedy_generated_data[i]],
      references=[[validation_data['target'][i]]]
  )
  greedy_bleu_results.append(greedy_bleu_result["score"])

sampling_bleu_results = []
for i, sentence in enumerate(zip(sampling_generated_data, validation_data)):
  sampling_bleu_result = sacrebleu_scorer.compute(
      predictions=[sampling_generated_data[i]],
      references=[[validation_data['target'][i]]]
  )
  sampling_bleu_results.append(sampling_bleu_result["score"])

greedy_bleu_mean = np.mean(greedy_bleu_results)
sampling_bleu_mean = np.mean(sampling_bleu_results)

print("Greedy BLEU mean", greedy_bleu_mean)
print("Sampling BLEU mean", sampling_bleu_mean)


Greedy BLEU mean 4.487229384024474
Sampling BLEU mean 3.8519618919934526


In [21]:
# BLEU references
greedy_bleu_results = []
for i, sentence in enumerate(zip(greedy_generated_data, validation_data)):
  greedy_bleu_result = sacrebleu_scorer.compute(
      predictions=[greedy_generated_data[i]],
      references=[validation_data['references'][i]]
  )
  greedy_bleu_results.append(greedy_bleu_result["score"])

sampling_bleu_results = []
for i, sentence in enumerate(zip(sampling_generated_data, validation_data)):
  sampling_bleu_result = sacrebleu_scorer.compute(
      predictions=[sampling_generated_data[i]],
      references=[validation_data['references'][i]]
  )
  sampling_bleu_results.append(sampling_bleu_result["score"])

greedy_bleu_mean = np.mean(greedy_bleu_results)
sampling_bleu_mean = np.mean(sampling_bleu_results)

print("Greedy BLEU mean", greedy_bleu_mean)
print("Sampling BLEU mean", sampling_bleu_mean)


Greedy BLEU mean 7.275324243167152
Sampling BLEU mean 5.910208284353547


In [22]:
### NIST score
from nltk.translate import nist_score

reference = 'un test simple'.split()
reference1 = 'un test super simple'.split()
candidate = 'un test pas trop simple'.split()

print('Sentence 1-gram: %f' % nist_score.sentence_nist([reference, reference1], candidate, 1))
print('Sentence 2-gram: %f' % nist_score.sentence_nist([reference], candidate, 2))
print('Sentence 3-gram: %f' % nist_score.sentence_nist([reference], candidate, 3))
print('Sentence 4-gram: %f' % nist_score.sentence_nist([reference], candidate, 4))
print('Sentence 5-gram: %f' % nist_score.sentence_nist([reference], candidate, 5))

print('Corpus 1-gram: %f' % nist_score.corpus_nist([[reference, reference1]], [candidate], 1))
print('Corpus 2-gram: %f' % nist_score.corpus_nist([[reference, reference1]], [candidate], 2))
print('Corpus 3-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 3))
print('Corpus 4-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 4))
print('Corpus 5-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 5))

Sentence 1-gram: 1.084413
Sentence 2-gram: 0.950978
Sentence 3-gram: 0.950978
Sentence 4-gram: 0.950978
Sentence 5-gram: 0.950978
Corpus 1-gram: 1.084413
Corpus 2-gram: 1.084413
Corpus 3-gram: 0.950978
Corpus 4-gram: 0.950978
Corpus 5-gram: 0.950978


In [23]:
# NIST
from nltk.translate import nist_score

greedy_nist_results = []
for i, sentence in enumerate(zip(validation_data, greedy_generated_data)):
  reference = validation_data['target'][i].split()
  candidate = greedy_generated_data[i].split()
  
  if len(candidate) > 4:
    greedy_nist_result = nist_score.sentence_nist(reference, candidate)
    greedy_nist_results.append(greedy_nist_result)

sampling_nist_results = []
for i, sentence in enumerate(zip(validation_data, sampling_generated_data)):
  reference = validation_data['target'][i].split()
  candidate = sampling_generated_data[i].split()
  
  if len(candidate) > 4:
    sampling_nist_result = nist_score.sentence_nist(reference, candidate)
    sampling_nist_results.append(sampling_nist_result)

    
greedy_nist_mean = np.mean(greedy_nist_results)
sampling_nist_mean = np.mean(sampling_nist_results)

print("Greedy NIST mean", greedy_nist_mean)
print("Sampling NIST mean", sampling_nist_mean)

Greedy NIST mean 0.1418103358983944
Sampling NIST mean 0.12445356319803778


In [24]:
# METEOR
greedy_meteor_results = meteor_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target']
)

sampling_meteor_results = meteor_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target']
)

print("Greedy meteor", round(greedy_meteor_results['meteor'], 4))
print("Sampling meteor", round(sampling_meteor_results['meteor'], 4))

Greedy meteor 0.2878
Sampling meteor 0.2773


 ## Semantic (BERTscore, BLEURT, NUBIA)

In [17]:
import numpy as np

bleurt_scorer = load_metric('bleurt', 'bleurt-large-512')
bertscore_scorer = load_metric("bertscore")


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/bleurt-large-512/downloads/extracted/299e33e80b83c78cc60e485384c7804f6ec1fb36c2013c5078257c17a82719ca/bleurt-large-512.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


In [26]:
# BLEURT

greedy_bleurt_results = bleurt_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target']
)
sampling_bleurt_results = bleurt_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target']
)

In [27]:
# When sequence length is 512
greedy_bleurt_results

{'scores': [-1.2770847082138062,
  -0.4408656656742096,
  -0.9973040223121643,
  -0.3044547140598297,
  -0.9340764284133911,
  -0.20779259502887726,
  -0.9404125809669495,
  -0.2863156497478485,
  -0.2371816784143448,
  -0.5456162691116333,
  -0.791816234588623,
  0.13666687905788422,
  -1.0806347131729126,
  -0.4762459099292755,
  -0.045821741223335266,
  -0.7370389103889465,
  -0.7574183940887451,
  -0.2708752453327179,
  -0.9815006852149963,
  -1.1978214979171753,
  -0.5726593136787415,
  -1.0858386754989624,
  -1.2973229885101318,
  -1.1185492277145386,
  -0.3655611574649811,
  -0.18095804750919342,
  -0.43656250834465027,
  -0.8751556277275085,
  -0.8431039452552795,
  -0.3959346115589142,
  -1.1074514389038086,
  -0.7621461153030396,
  -0.5616394281387329,
  -1.0233179330825806,
  -0.5053853988647461,
  -0.3684000074863434,
  -0.8658005595207214,
  -0.7173128128051758,
  -0.8189133405685425,
  -0.24187947809696198,
  -0.4069876968860626,
  -0.8410391807556152,
  -0.64469605684280

In [29]:
# When seq len = 512
sampling_bleurt_results

{'scores': [-0.8802810311317444,
  -0.06412793695926666,
  -0.9346972107887268,
  -0.5087388753890991,
  -0.5711228847503662,
  -0.4709087312221527,
  -0.9070395827293396,
  -0.1302691251039505,
  -0.4039420187473297,
  -1.120941162109375,
  -1.2281965017318726,
  -0.11380256712436676,
  -1.150852084159851,
  -0.737303614616394,
  -0.35711100697517395,
  -0.8295895457267761,
  -1.0745562314987183,
  -0.23273952305316925,
  -1.0263806581497192,
  -1.0567494630813599,
  0.022011980414390564,
  -1.0376861095428467,
  -1.1074973344802856,
  -0.833763062953949,
  0.057717666029930115,
  -0.031971290707588196,
  -0.48642292618751526,
  -0.5207933783531189,
  -0.922050952911377,
  -0.2106538563966751,
  -1.2566887140274048,
  -0.8833947777748108,
  -0.7845109105110168,
  -0.3576735556125641,
  -0.7870051264762878,
  -0.9895642399787903,
  -0.5525789260864258,
  -0.4138623774051666,
  -1.2114332914352417,
  -0.6262792348861694,
  -0.1878315955400467,
  -0.7077574133872986,
  -0.609825074672699

In [31]:
greedy_bleurt_mean = np.mean(greedy_bleurt_results['scores'])
sampling_bleurt_mean = np.mean(sampling_bleurt_results['scores'])

print("Greedy bleurt mean", greedy_bleurt_mean)
print("Sampling bleurt mean", sampling_bleurt_mean)

Greedy bleurt mean -0.59893669127904
Sampling bleurt mean -0.702536350021407


In [32]:
print("Min greedy value: ", np.min(greedy_bleurt_results.get('scores')))
print("Max greedy value: ", np.max(greedy_bleurt_results.get('scores')))

Min greedy value:  -1.4401649236679077
Max greedy value:  0.47240006923675537


In [33]:
print("Min sampling value: ", np.min(sampling_bleurt_results.get('scores')))
print("Max sampling value: ", np.max(sampling_bleurt_results.get('scores')))

Min sampling value:  -1.4147217273712158
Max sampling value:  0.26421603560447693


In [18]:
# BERTscore

greedy_bertscore_results = bertscore_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target'],
    lang="en"
)

sampling_bertscore_results = bertscore_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target'],
    lang="en"
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
greedy_bertscore_results

{'f1': [0.8589634299278259,
  0.8778308629989624,
  0.8833815455436707,
  0.8942566514015198,
  0.8819716572761536,
  0.9032708406448364,
  0.857177197933197,
  0.8852366805076599,
  0.8454146385192871,
  0.8891057372093201,
  0.8542487621307373,
  0.8911428451538086,
  0.8309435248374939,
  0.8896595239639282,
  0.8920561671257019,
  0.8688803911209106,
  0.8953477740287781,
  0.8903221487998962,
  0.8751536011695862,
  0.8650606870651245,
  0.8770608901977539,
  0.8681526780128479,
  0.8640832304954529,
  0.8550593852996826,
  0.8676882982254028,
  0.9124632477760315,
  0.8984365463256836,
  0.8693664073944092,
  0.8768914937973022,
  0.9189810752868652,
  0.8542450070381165,
  0.8817676305770874,
  0.8868319988250732,
  0.845735490322113,
  0.8619670271873474,
  0.8911656737327576,
  0.8629369139671326,
  0.8679993748664856,
  0.8551411032676697,
  0.8700724244117737,
  0.8961684703826904,
  0.8812351226806641,
  0.9088289141654968,
  0.8675143122673035,
  0.860284149646759,
  0.877

In [20]:
greedy_bertscore_f1_mean = np.mean(greedy_bertscore_results['f1'])
greedy_bertscore_precision_mean = np.mean(greedy_bertscore_results['precision'])
greedy_bertscore_recall_mean = np.mean(greedy_bertscore_results['recall'])

sampling_bertscore_f1_mean = np.mean(sampling_bertscore_results['f1'])
sampling_bertscore_precision_mean = np.mean(sampling_bertscore_results['precision'])
sampling_bertscore_recall_mean = np.mean(sampling_bertscore_results['recall'])

print("Greedy F1 mean", round(greedy_bertscore_f1_mean, 4))
print("Greedy precision mean", round(greedy_bertscore_precision_mean, 4))
print("Greedy recall mean", round(greedy_bertscore_recall_mean, 4))

print("Sampling F1 mean", round(sampling_bertscore_f1_mean, 4))
print("Sampling precision mean", round(sampling_bertscore_precision_mean, 4))
print("Sampling recall mean", round(sampling_bertscore_recall_mean, 4))


Greedy F1 mean 0.8757
Greedy precision mean 0.8603
Greedy recall mean 0.8918
Sampling F1 mean 0.8715
Sampling precision mean 0.8534
Sampling recall mean 0.8905


In [21]:
import os
os.chdir('nubia')

In [22]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.7MB 4.7MB/s 
     |████████████████████████████████| 184kB 24.9MB/s 
     |████████████████████████████████| 2.5MB 25.2MB/s 
     |████████████████████████████████| 153kB 39.9MB/s 
     |████████████████████████████████| 133kB 42.5MB/s 
     |████████████████████████████████| 1.9MB 36.2MB/s 
     |████████████████████████████████| 215kB 35.7MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 112kB 50.9MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 7.7MB 43.6MB/s 
     |████████████████████████████████| 962kB 32.4MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 3.2MB 27.7MB/s 
     |████████████████████████████████| 645kB 34.7MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=a46a2d375dcba7076d05b57d8b16d1479a46e4338fe56a1e7afa48d151b3382a
  Stored in directory: 

In [23]:
from nubia_score import Nubia
n = Nubia()

1042301B [00:00, 2409340.83B/s]
456318B [00:00, 1573176.86B/s]
100%|██████████| 548118077/548118077 [00:12<00:00, 42879765.50B/s]


In [24]:
greedy_nubia_contradiction = []
greedy_nubia_grammar_hyp = []
greedy_nubia_grammar_ref = []
greedy_nubia_logical_agreement = []
greedy_nubia_semantic_relation = []
greedy_nubia_score = []

for i, sentence in enumerate(zip(validation_data, greedy_generated_data)):
  greedy_nubia_result = n.score(
      greedy_generated_data[i],
      validation_data['target'][i],
      verbose=True, 
      get_features=True
  )

  greedy_nubia_contradiction.append(greedy_nubia_result.get('features').get('contradiction'))
  greedy_nubia_grammar_hyp.append(greedy_nubia_result.get('features').get('grammar_hyp'))
  greedy_nubia_grammar_ref.append(greedy_nubia_result.get('features').get('grammar_ref'))
  greedy_nubia_logical_agreement.append(greedy_nubia_result.get('features').get('logical_agreement'))
  greedy_nubia_semantic_relation.append(greedy_nubia_result.get('features').get('semantic_relation'))

  greedy_nubia_score.append(greedy_nubia_result.get('nubia_score'))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

NUBIA score: 0.33231598790401373/1.0
Semantic relation: 1.863703727722168/5.0
Percent chance of contradiction: 13.724268972873688%
Percent chance of irrelevancy or new information: 53.62043380737305%
Percent chance of logical agreement: 32.655295729637146%

NUBIA score: 0.1016522342864592/1.0
Semantic relation: 1.0973424911499023/5.0
Percent chance of contradiction: 0.1423073816113174%
Percent chance of irrelevancy or new information: 99.76428151130676%
Percent chance of logical agreement: 0.09340684046037495%

NUBIA score: 0.08841716457939686/1.0
Semantic relation: 2.3595776557922363/5.0
Percent chance of contradiction: 0.1282872981391847%
Percent chance of irrelevancy or new information: 99.6615469455719%
Percent chance of logical agreement: 0.21016583777964115%

NUBIA score: 0.11721053270565873/1.0
Semantic relation: 3.014341354370117/5.0
Percent chance of contradiction: 0.4143384285271168%
Percent chance

In [25]:
import numpy as np

greedy_nubia_contradiction
greedy_nubia_grammar_hyp
greedy_nubia_grammar_ref
greedy_nubia_logical_agreement
greedy_nubia_semantic_relation
greedy_nubia_score

greedy_nubia_contradiction_mean = np.mean(greedy_nubia_contradiction)
greedy_nubia_grammar_hyp_mean = np.mean(greedy_nubia_grammar_hyp)
greedy_nubia_grammar_ref_mean = np.mean(greedy_nubia_grammar_ref)
greedy_nubia_logical_agreement_mean = np.mean(greedy_nubia_logical_agreement)
greedy_nubia_semantic_relation_mean = np.mean(greedy_nubia_semantic_relation)
greedy_nubia_score_mean = np.mean(greedy_nubia_score)

print("Greedy nubia_contradiction mean", round(greedy_nubia_contradiction_mean, 4))
print("Greedy nubia_grammar_hyp mean", round(greedy_nubia_grammar_hyp_mean, 4))
print("Greedy nubia_grammar_ref mean", round(greedy_nubia_grammar_ref_mean, 4))
print("Greedy nubia_logical_agreement mean", round(greedy_nubia_logical_agreement_mean, 4))
print("Greedy nubia_semantic_relation mean", round(greedy_nubia_semantic_relation_mean, 4))
print("Greedy nubia_score mean", round(greedy_nubia_score_mean, 4))

Greedy nubia_contradiction mean 23.6192
Greedy nubia_grammar_hyp mean 4.5859
Greedy nubia_grammar_ref mean 4.1603
Greedy nubia_logical_agreement mean 19.3532
Greedy nubia_semantic_relation mean 2.4657
Greedy nubia_score mean 0.206


In [26]:
sampling_nubia_contradiction = []
sampling_nubia_grammar_hyp = []
sampling_nubia_grammar_ref = []
sampling_nubia_logical_agreement = []
sampling_nubia_semantic_relation = []
sampling_nubia_score = []

for i, sentence in enumerate(zip(validation_data, sampling_generated_data)):
  sampling_nubia_result = n.score(
      sampling_generated_data[i],
      validation_data['target'][i],
      verbose=True, 
      get_features=True
  )

  sampling_nubia_contradiction.append(sampling_nubia_result.get('features').get('contradiction'))
  sampling_nubia_grammar_hyp.append(sampling_nubia_result.get('features').get('grammar_hyp'))
  sampling_nubia_grammar_ref.append(sampling_nubia_result.get('features').get('grammar_ref'))
  sampling_nubia_logical_agreement.append(sampling_nubia_result.get('features').get('logical_agreement'))
  sampling_nubia_semantic_relation.append(sampling_nubia_result.get('features').get('semantic_relation'))

  sampling_nubia_score.append(sampling_nubia_result.get('nubia_score'))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

NUBIA score: 0.07318379289892762/1.0
Semantic relation: 2.5636510848999023/5.0
Percent chance of contradiction: 1.7568482086062431%
Percent chance of irrelevancy or new information: 96.43692970275879%
Percent chance of logical agreement: 1.8062211573123932%

NUBIA score: 0.13777046309314844/1.0
Semantic relation: 1.6847083568572998/5.0
Percent chance of contradiction: 27.05841064453125%
Percent chance of irrelevancy or new information: 68.81682872772217%
Percent chance of logical agreement: 4.124752059578896%

NUBIA score: 0.08636894362361558/1.0
Semantic relation: 2.8490636348724365/5.0
Percent chance of contradiction: 0.5472718272358179%
Percent chance of irrelevancy or new information: 99.04224872589111%
Percent chance of logical agreement: 0.41048931889235973%

NUBIA score: 0.1466817058435378/1.0
Semantic relation: 1.7966331243515015/5.0
Percent chance of contradiction: 95.23725509643555%
Percent chance 

In [27]:
sampling_nubia_contradiction
sampling_nubia_grammar_hyp
sampling_nubia_grammar_ref
sampling_nubia_logical_agreement
sampling_nubia_semantic_relation
sampling_nubia_score

sampling_nubia_contradiction_mean = np.mean(sampling_nubia_contradiction)
sampling_nubia_grammar_hyp_mean = np.mean(sampling_nubia_grammar_hyp)
sampling_nubia_grammar_ref_mean = np.mean(sampling_nubia_grammar_ref)
sampling_nubia_logical_agreement_mean = np.mean(sampling_nubia_logical_agreement)
sampling_nubia_semantic_relation_mean = np.mean(sampling_nubia_semantic_relation)
sampling_nubia_score_mean = np.mean(sampling_nubia_score)

print("Sampling nubia_contradiction mean", round(sampling_nubia_contradiction_mean, 4))
print("Sampling nubia_grammar_hyp mean", round(sampling_nubia_grammar_hyp_mean, 4))
print("Sampling nubia_grammar_ref mean", round(sampling_nubia_grammar_ref_mean, 4))
print("Sampling nubia_logical_agreement mean", round(sampling_nubia_logical_agreement_mean, 4))
print("Sampling nubia_semantic_relation mean", round(sampling_nubia_semantic_relation_mean, 4))
print("Sampling nubia_score mean", round(sampling_nubia_score_mean, 4))

Sampling nubia_contradiction mean 25.3043
Sampling nubia_grammar_hyp mean 4.5859
Sampling nubia_grammar_ref mean 4.4751
Sampling nubia_logical_agreement mean 19.624
Sampling nubia_semantic_relation mean 2.216
Sampling nubia_score mean 0.1714


## Diversity (MSTTR, Distinct 1/2/3, Unique 1/2/3, Entropy 1/2/3)


In [28]:
# Mean segmental Type-Token Ratio (MSTTR)
from lexical_diversity import lex_div as ld

greedy_msttr_results = []
for i, sentence in enumerate(greedy_generated_data):
  greedy_flt = ld.flemmatize(greedy_generated_data[i])
  greedy_msttr_result = ld.msttr(greedy_flt)
  greedy_msttr_results.append(greedy_msttr_result)

sampling_msttr_results = []
for i, sentence in enumerate(sampling_generated_data):
  sampling_flt = ld.flemmatize(sampling_generated_data[i])
  sampling_msttr_result = ld.msttr(sampling_flt)
  sampling_msttr_results.append(sampling_msttr_result)


greedy_msttr_mean = np.mean(greedy_msttr_results)
sampling_msttr_mean = np.mean(sampling_msttr_results)

print("Greedy MSTTR mean", greedy_msttr_mean)
print("Sampling MSTTR ", sampling_msttr_mean)

Greedy MSTTR mean 0.8123501277004496
Sampling MSTTR  0.8401079316455407


In [29]:
# Distinct-n
from distinct_n import distinct_n_corpus_level

greedy_distinct_1_results = distinct_n_corpus_level(greedy_generated_data, 1)
sampling_distinct_1_results = distinct_n_corpus_level(sampling_generated_data, 1)

greedy_distinct_2_results = distinct_n_corpus_level(greedy_generated_data, 2)
sampling_distinct_2_results = distinct_n_corpus_level(sampling_generated_data, 2)

greedy_distinct_3_results = distinct_n_corpus_level(greedy_generated_data, 3)
#sampling_distinct_3_results = distinct_n_corpus_level(sampling_generated_data, 3)

print("Greedy distict 1-gram", greedy_distinct_1_results)
print("Greedy distict 2-gram", greedy_distinct_2_results)
print("Greedy distict 3-gram", greedy_distinct_3_results)

print("Sampling distict 1-gram", sampling_distinct_1_results)
print("Sampling distict 2-gram", sampling_distinct_2_results)
#print("Sampling distict 3-gram", sampling_distinct_3_results)

Greedy distict 1-gram 0.20670604038332874
Greedy distict 2-gram 0.5998772549627058
Greedy distict 3-gram 0.7883199691513992
Sampling distict 1-gram 0.18902230696770012
Sampling distict 2-gram 0.5973102080858873


In [30]:
# Unique-n

import nltk

greedy_document = []
for sentence in greedy_generated_data:
  sentence = sentence.split()
  for word in sentence:
    greedy_document.append(word)

sampling_document = []
for sentence in sampling_generated_data:
  sentence = sentence.split()
  for word in sentence:
    sampling_document.append(word)

greedy_unigram_finder = nltk.FreqDist(greedy_document)
greedy_bigram_finder = nltk.FreqDist(nltk.bigrams(greedy_document))
greedy_trigram_finder = nltk.FreqDist(nltk.trigrams(greedy_document))

sampling_unigram_finder = nltk.FreqDist(sampling_document)
sampling_bigram_finder = nltk.FreqDist(nltk.bigrams(sampling_document))
sampling_trigram_finder = nltk.FreqDist(nltk.trigrams(sampling_document))

def unique_finder(finder):
  counter = 0
  for occurence in range(len(finder)):
    item = finder.popitem()
    if item[1] == 1:
      counter += 1
  return counter

greedy_uni_count = unique_finder(greedy_unigram_finder)
greedy_bi_count = unique_finder(greedy_bigram_finder)
greedy_tri_count = unique_finder(greedy_trigram_finder)

sampling_uni_count = unique_finder(sampling_unigram_finder)
sampling_bi_count = unique_finder(sampling_bigram_finder)
sampling_tri_count = unique_finder(sampling_trigram_finder)

print("Greedy unique unigrams", greedy_uni_count)
print("Greedy unique bigrams", greedy_bi_count)
print("Greedy unique trigrams", greedy_tri_count)

print("Sampling unique unigrams", sampling_uni_count)
print("Sampling unique bigrams", sampling_bi_count)
print("Sampling unique trigrams", sampling_tri_count)

Greedy unique unigrams 1042
Greedy unique bigrams 7752
Greedy unique trigrams 16037
Sampling unique unigrams 1467
Sampling unique bigrams 10694
Sampling unique trigrams 21451


In [31]:
# Entropy-n
from scipy import stats

greedy_unigram_finder = nltk.FreqDist(greedy_document)
greedy_bigram_finder = nltk.FreqDist(nltk.bigrams(greedy_document))
greedy_trigram_finder = nltk.FreqDist(nltk.trigrams(greedy_document))

sampling_unigram_finder = nltk.FreqDist(sampling_document)
sampling_bigram_finder = nltk.FreqDist(nltk.bigrams(sampling_document))
sampling_trigram_finder = nltk.FreqDist(nltk.trigrams(sampling_document))

greedy_unigram_probabilities = [v for k,v in greedy_unigram_finder.items()]
greedy_bigram_probabilities = [v for k,v in greedy_bigram_finder.items()]
greedy_trigram_probabilities = [v for k,v in greedy_trigram_finder.items()]

sampling_unigram_probabilities = [v for k,v in sampling_unigram_finder.items()]
sampling_bigram_probabilities = [v for k,v in sampling_bigram_finder.items()]
sampling_trigram_probabilities = [v for k,v in sampling_trigram_finder.items()]

greedy_entropy_1 = stats.entropy(greedy_unigram_probabilities)
greedy_entropy_2 = stats.entropy(greedy_bigram_probabilities)
greedy_entropy_2 = stats.entropy(greedy_trigram_probabilities)

sampling_entropy_1 = stats.entropy(sampling_unigram_probabilities)
sampling_entropy_2 = stats.entropy(sampling_bigram_probabilities)
sampling_entropy_2 = stats.entropy(sampling_trigram_probabilities)

print("Greedy entropy-1:", greedy_entropy_1)
print("Greedy entropy-2:", greedy_entropy_2)
#print("Greedy entropy-3:", greedy_entropy_3)

print("Sampling entropy-1:", sampling_entropy_1)
print("Sampling entropy-2:", sampling_entropy_2)
#print("Sampling entropy-3:", sampling_entropy_3)

Greedy entropy-1: 5.611564668155908
Greedy entropy-2: 9.610578681096444
Sampling entropy-1: 5.829582327959128
Sampling entropy-2: 9.951091606394833
